# CS486 - Artificial Intelligence
## Lesson 12 - Expectimax

Minimax is used to reason about the outcomes of adversarial decisions. We can use the same strategy to reason about the outcomes of chance.  

In [ ]:
import helpers
from aima.games import *
from aima.notebook import psource

## Draw HiLo

Let's use expectimax to play Draw HiLo, is card game where you guess if successive cards will be higher or lower than the previes card. First, let's walk through an AIMA implementation of the game:

In [ ]:
from random import randint

class HiLo(StochasticGame):
    def __init__(self):
        self.payouts = {
            1:  {-1: 0,    1: 1},
            2:  {-1: 12,   1: 1.1},
            3:  {-1: 5,    1: 1.2},
            4:  {-1: 4,    1: 1.3},
            5:  {-1: 3,    1: 1.4},
            6:  {-1: 2,    1: 1.5},
            7:  {-1: 1.8,  1: 1.8},
            8:  {-1: 1.5,  1: 2},
            9:  {-1: 1.4,  1: 3},
            10: {-1: 1.3,  1: 4},
            11: {-1: 1.2,  1: 5},
            12: {-1: 1.1,  1: 12},
            13: {-1: 1,    1: 0}
        }
        
        self.initial = StochasticGameState(
            to_move=1,
            utility=1,
            board=[randint(2,12)],
            moves=[-1,1], 
            chance=None
        )
    
    def chances(self,state):
        return list(range(1,14))

    def probability(self,chance):
        return 1/13
    
    def outcome(self,state,chance):
        return StochasticGameState(
            to_move=1,
            utility=state.utility,
            board=state.board.copy(),
            moves=state.moves, 
            chance=chance
        )

    def result(self,state,action):
        card = state.board[-1]
        draw = state.chance
        hilo = (draw>card)-(draw<card)

        if hilo == 0:
            utility = state.utility
        elif hilo == action:
            utility = state.utility*self.payouts[card][hilo]
        else:
            utility = -1
        
        return StochasticGameState(
            to_move=1,
            utility=utility,
            board=state.board + [draw],
            moves=state.moves, 
            chance=None
        )
    
    def utility(self,state,player):
        return state.utility
    
    def actions(self,state):
        return state.moves
    
    def terminal_test(self,state):
        return len(state.board) == 5 or state.utility < 0
    
    def display(self,state):
        print("Cards:", ", ".join(map(str,state.board)))

## Let's Play!

First, let's see how a random player does in HiLo:

In [ ]:
HiLo().play_game(random_player)

Random doesn't seem very effective. What do you think the odds are that the random player wins a 5-round game? Now let's see how a human does:

In [ ]:
HiLo().play_game(query_player)

## Expectimax Player

How exactly do we decide which card to play? Well, we have an intuition for how many cards are higher and play accordingly. That's exactly what the expectimax algorithm gives us. 

Let's code up an expectimax player and see how they do:

In [ ]:
def expectimax_player(game,state):
    def max_value(state):
        v = -infinity
        for a in game.actions(state):
            v = max(v, chance_node(state, a))
        return v

    def chance_node(state, action):
        sum_chances = 0
        num_chances = len(game.chances(state))

        for chance in game.chances(state):
            res_state = game.outcome(state, chance)
            res_state = game.result(res_state, action)

            if game.terminal_test(res_state):
                util = game.utility(res_state, res_state.to_move)
            else:
                util = max_value(res_state)

            sum_chances += util * game.probability(chance)

        return sum_chances / num_chances

    return argmax(game.actions(state),
              key=lambda a: chance_node(state, a), default=None)

Let's see how our expectimax player performs:

In [ ]:
HiLo().play_game(expectimax_player)

To get a better idea of the math behind a particular decision, compute the decision for case where a 5 is up, assuming it's the final draw of the game.